In [2]:
from __future__ import print_function
from __future__ import division
import pandas as pd
import gzip
import time
import os, sys, time
import collections
from collections import Counter
import itertools

# Numerical manipulation libraries.
import numpy as np

#Visualization
import matplotlib
%matplotlib inline

In [11]:
#Read original data
with open('neighbors.json', 'r') as f_:
     data = json.load(f_)

In [54]:
#Convert to a pandas df with item and similar items
df = pd.DataFrame(data.items(), columns=['item', 'similar'])

In [55]:
df.head()

,item,similar
0,B004JIL6G6,"[B002445NOS, B003JBYYNG, B0049DJPUU, B004K1EZD..."
1,B003JBYYNG,"[B004JIL6G6, B0049DJPUU, B004K1EZD8, B001R6W4K..."
2,B0049DJPUU,"[B004JIL6G6, B003JBYYNG, B004K1EZD8, B001R6W4K..."
3,B004K1EZD8,"[B00ATWUE2S, B00H5HFERA, B00IVFCGIA, B006O6EH4..."
4,B001R6W4KA,"[B001DZMITW, B0015KVXW8, B0036AWTDM, B0042ES4O..."


In [70]:
import random
from collections import Counter
allItems = df['item'].tolist()
numSelections = 3

In [145]:
#Function to read the similar items to all items in userItems (list of asins picked by user, and output the most common asins within those)
def findSimilars(df,userItems):
    similarItems = []
    similarItemsDf = df[df['item'].isin(userItems)]['similar'].reset_index(drop = True)
#     similarItems = [item for sublist in similarItemsDf for item in sublist]
    for i in range(similarItemsDf.shape[0]):
        for item in similarItemsDf[i]:
            similarItems.append(item)
#     print(len(similarItems),similarItems)
    a = Counter(similarItems)
    return a

In [76]:
#Test against all items included in MF, irrespective of category.
userItems = random.sample(allItems, numSelections)
print(userItems)
a = findSimilars(df,userItems)
a.most_common(5)

['B0038LYVVM', 'B006ZGX61S', 'B000063S5P']
60 ['B004JIL6G6', 'B003JBYYNG', 'B0049DJPUU', 'B004K1EZD8', 'B001R6W4KA', 'B002I4OTA8', 'B001LF3T6K', 'B004LCXH3K', 'B00143XOGO', 'B006GDLDSI', 'B0006PKZBI', 'B0007QO4FY', 'B001KXML0I', 'B0015GMFNS', 'B001W95O04', 'B000F5FK2S', 'B004VMT90A', 'B001W2WKS0', 'B000NSW0LQ', 'B001GZXYFG', 'B003AWYJRA', 'B00569DOTO', 'B003B7Q5YY', 'B0023TCKEK', 'B00374BIRK', 'B004JIL6G6', 'B003JBYYNG', 'B0049DJPUU', 'B004K1EZD8', 'B001R6W4KA', 'B002I4OTA8', 'B001LF3T6K', 'B004LCXH3K', 'B00143XOGO', 'B006GDLDSI', 'B0006PKZBI', 'B0007QO4FY', 'B001KXML0I', 'B0015GMFNS', 'B001W95O04', 'B004JIL6G6', 'B003JBYYNG', 'B0049DJPUU', 'B004K1EZD8', 'B001R6W4KA', 'B002I4OTA8', 'B001LF3T6K', 'B004LCXH3K', 'B00143XOGO', 'B006GDLDSI', 'B0006PKZBI', 'B0007QO4FY', 'B001KXML0I', 'B0015GMFNS', 'B001W95O04', 'B000F5FK2S', 'B004VMT90A', 'B001W2WKS0', 'B000NSW0LQ', 'B001GZXYFG']


[('B004JIL6G6', 3),
 ('B003JBYYNG', 3),
 ('B0049DJPUU', 3),
 ('B004K1EZD8', 3),
 ('B001R6W4KA', 3)]

In [77]:
#Read product metadata
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

dfMeta = getDF('meta_Toys_and_Games.json.gz')

In [79]:
dfMeta[dfMeta['asin'].isin(userItems)]

,asin,description,title,price,salesRank,imUrl,brand,categories,related
7642,B000063S5P,"mint in sealed box , dent on one side and smal...",MicroMachines Action Fleet Millenium Falcon,68.99,{'Toys & Games': 152490},http://ecx.images-amazon.com/images/I/410BVMZA...,NaN,"[[Toys & Games, Action Figures & Statues, Play...","{'also_viewed': ['B004S6DXKG', 'B0013J5QQ0', '..."
152337,B0038LYVVM,24 Dinosaur Stampers. Add a little prehistoric...,Assorted Dinosaur Stampers (24 PCS),6.31,{'Toys & Games': 1086},http://ecx.images-amazon.com/images/I/41tS5gYN...,OTC,"[[Toys & Games, Arts & Crafts, Printing & Stam...","{'also_bought': ['B0023TCKEK', 'B001DN8I0M', '..."
239168,B006ZGX61S,The cuteness of Lalaloopsy is now in the palm...,MGA Mini Lalaloopsy Playsets - Camping with Su...,NaN,{'Toys & Games': 159049},http://ecx.images-amazon.com/images/I/41l1f-4x...,NaN,"[[Toys & Games, Dolls & Accessories, Dolls]]","{'also_bought': ['B006ZGX60O', 'B004ZK6LCC', '..."


In [80]:
#Merge similar items and category fields of metdata to allow picking items from a subcategory
dfComb = pd.merge(df,dfMeta[['asin','categories']],left_on = ['item'],right_on = ['asin'], how = 'inner')

In [107]:
#Create a column containing the subcategory within toys and games
dfComb['subCateg'] = dfComb['categories'].map(lambda x:x[0][1] if len(x[0]) > 1 else x[0][0])
dfCombSub = dfComb[dfComb['subCateg'] == 'Baby & Toddler Toys']
dfCombSub.head()

,item,similar,asin,categories,subCateg
14,B001W95O04,"[B004JIL6G6, B003JBYYNG, B0049DJPUU, B004K1EZD...",B001W95O04,"[[Toys & Games, Baby & Toddler Toys, Bath Toys]]",Baby & Toddler Toys
69,B00000J4XU,"[B004JIL6G6, B003JBYYNG, B0049DJPUU, B004K1EZD...",B00000J4XU,"[[Toys & Games, Baby & Toddler Toys, Rattles]]",Baby & Toddler Toys
74,B004TS8HF4,"[B004JIL6G6, B003JBYYNG, B0049DJPUU, B004K1EZD...",B004TS8HF4,"[[Toys & Games, Baby & Toddler Toys, Bath Toys]]",Baby & Toddler Toys
104,B001IEYXAA,"[B004560BPY, B00428LIZ2, B00068Q7LC, B00428LJ0...",B001IEYXAA,"[[Toys & Games, Baby & Toddler Toys, Music & S...",Baby & Toddler Toys
128,B000VO3GME,"[B00005RF5G, B004H1V5S4, B0006IRTU0, B000W3XEQ...",B000VO3GME,"[[Toys & Games, Baby & Toddler Toys, Hammering...",Baby & Toddler Toys


In [114]:
#List of all items in baby and toddler subcategory
babyItems = dfCombSub['item'].tolist()
babyItems[:5]

['B001W95O04', 'B00000J4XU', 'B004TS8HF4', 'B001IEYXAA', 'B000VO3GME']

In [153]:
#Find most common similar items for a randomly sampled set of items in babyItems (numSelections is the number of items picked by the user)
babyUserItems = random.sample(babyItems, numSelections)
print(babyUserItems)
# for i in range(len(babyUserItems)):
#     print(dfCombSub[dfCombSub['asin'] == babyUserItems[i]])
a = findSimilars(dfCombSub,babyUserItems)
# a.most_common(5)
similarList = []
for l in a.most_common(5):
    similarList.append(l[0])
similarList

['B00772360K', 'B00004VX1G', 'B008IZH2M8']


['B004JIL6G6', 'B003JBYYNG', 'B0049DJPUU', 'B004K1EZD8', 'B001R6W4KA']

In [150]:
#Metadata for randomlu picked items
dfMeta[dfMeta['asin'].isin(babyUserItems)]

,asin,description,title,price,salesRank,imUrl,brand,categories,related
147461,B00336TPJ0,"Dip, turn and spin! Cute octopus sprinkle toy...",ALEX&reg; Toys - Bathtime Fun Spin &amp; Spray...,14.43,{'Toys & Games': 132175},http://ecx.images-amazon.com/images/I/51g5C0ae...,Alex Toys,"[[Toys & Games, Baby & Toddler Toys, Bath Toys]]","{'also_bought': ['B000G2JQB6', 'B005IFW3JS', '..."
238228,B006WAYHN8,12 Chocolate Easter Bunny Rubber Duckies. Look...,12 ct - Chocolate Easter Bunny Rubber Duckies,7.29,{'Toys & Games': 166134},http://ecx.images-amazon.com/images/I/41WThIWw...,OTC,"[[Toys & Games, Baby & Toddler Toys, Bath Toys]]","{'also_bought': ['B000E5VCGW', 'B00BRS7QU6', '..."
264293,B008MTYU7U,Little Tikes PopTunes are musical toy instrume...,Little Tikes PopTunes Keyboard,23.99,{'Toys & Games': 8516},http://ecx.images-amazon.com/images/I/41Npm72M...,Little Tikes,"[[Toys & Games, Baby & Toddler Toys, Music & S...","{'also_bought': ['B008MTYU84', 'B008MTYTWQ', '..."


In [151]:
#Metadata for most similar items
dfMeta[dfMeta['asin'].isin(similarList)]

,asin,description,title,price,salesRank,imUrl,brand,categories,related
113892,B001R6W4KA,,Baby Stella Peach Doll,27.79,NaN,http://ecx.images-amazon.com/images/I/31iOVPC-...,Manhattan Toy,[[Toys & Games]],"{'also_bought': ['B00BR3E226', 'B0015KVXW8', '..."
161216,B003JBYYNG,5.5 in This grey stuffed squirrel is made of t...,Shasta Squirrel 5&quot;,11.92,{'Toys & Games': 83179},http://ecx.images-amazon.com/images/I/41-KwoHf...,Douglas Cuddle Toys,"[[Toys & Games, Stuffed Animals & Plush, Anima...","{'also_bought': ['B000GBMHCC', 'B000GBMHC2', '..."
181153,B0049DJPUU,Barbie's full name is Barbara Millicent Robert...,Black Evening Gown with Shawl 2010 Holiday Bar...,6.50,{'Toys & Games': 219095},http://ecx.images-amazon.com/images/I/41uG07D%...,NaN,"[[Toys & Games, Dolls & Accessories, Doll Acce...","{'also_bought': ['B002C9DH3E', 'B005M337Z0', '..."
189930,B004JIL6G6,Transform this velvet-flocked tree into a fun ...,Creativity For Kids Shrinky Dinks Christmas Tree,11.24,{'Toys & Games': 84242},http://ecx.images-amazon.com/images/I/61T2VzyZ...,Creativity for Kids,"[[Toys & Games, Arts & Crafts, Craft Kits, Jew...","{'also_bought': ['B007QNDWB8', 'B000UEDTWC', '..."
190436,B004K1EZD8,,ALEX&reg; Toys - Craft M&amp;M'S Candy Wrapper...,2.99,{'Toys & Games': 132308},http://ecx.images-amazon.com/images/I/61y8yvFu...,Alex Toys,"[[Toys & Games, Arts & Crafts, Craft Kits, Jew...","{'also_bought': ['B004K1EZ92', 'B004K1EZCO', '..."


In [154]:
#Doing a count of how many times each product occurs in the similar items list within baby and toddler toys
similarItems = []
similarItemsDf = dfCombSub[dfCombSub['item'].isin(babyItems)]['similar'].reset_index(drop = True)
#     similarItems = [item for sublist in similarItemsDf for item in sublist]
for i in range(similarItemsDf.shape[0]):
    for item in similarItemsDf[i]:
        similarItems.append(item)
#     print(len(similarItems),similarItems)
a = Counter(similarItems)
a.most_common(20)

[('B004JIL6G6', 1162),
 ('B003JBYYNG', 1160),
 ('B0049DJPUU', 1159),
 ('B004K1EZD8', 1158),
 ('B001R6W4KA', 1155),
 ('B002I4OTA8', 1154),
 ('B001LF3T6K', 1153),
 ('B004LCXH3K', 1152),
 ('B00143XOGO', 1150),
 ('B006GDLDSI', 1144),
 ('B0006PKZBI', 1141),
 ('B0007QO4FY', 1137),
 ('B001KXML0I', 1131),
 ('B0015GMFNS', 1128),
 ('B001W95O04', 1120),
 ('B000F5FK2S', 1112),
 ('B004VMT90A', 1088),
 ('B001W2WKS0', 1062),
 ('B000NSW0LQ', 1018),
 ('B001GZXYFG', 928)]